In [1]:
from dotenv import load_dotenv
from crewai import Crew
from tasks import MemoryTasks
from agents import LTMAgents
load_dotenv()


CLEARING MEMORY


True

In [2]:
user_context = "I want to create a meal plan for myself for lunch but Im allergic to fish. Im also a vegetarian?"
subsequent_request = 'can you provide a meal plan for me?'

In [3]:

tasks = MemoryTasks()
agents = LTMAgents()

# create agents
context_agent = agents.context_agent()
memory_agent = agents.memory_agent()
retrieval_agent = agents.retrieval_agent()

# create tasks
context_task = tasks.determine_memory_task(context_agent, user_context)
memory_task = tasks.store_memory(memory_agent)
retrieve_and_combine_relevant_memories = tasks.retrieve_and_combine_relevant_memories(retrieval_agent, subsequent_request)

memory_task.context = [context_task]
    
crew = Crew(
    agents=[
        context_agent, 
        memory_agent,
        retrieval_agent
    ],
    tasks=[
        context_task,
        memory_task,
        retrieve_and_combine_relevant_memories
    ]
)

In [4]:
result = crew.kickoff()



> Entering new CrewAgentExecutor chain...
Thought: I now can analyze the most recent message provided by the user to identify any relevant details that should be recorded in our knowledge base.

Final Answer: Allergy: Fish, Dietary Preference: Vegetarian

> Finished chain.


> Entering new CrewAgentExecutor chain...
Thought: Before committing any memory to the database, it's crucial to ensure that the memory does not already exist to prevent duplicates. I need to verify if the details provided by the context_specialist, which are "Allergy: Fish" and "Dietary Preference: Vegetarian", are already stored in the database.

Action: Ask question to co-worker
Action Input: {"coworker": "Retrieval Specialist", "question": "Do these memories already exist in the database?", "context": "The memories to check are related to an Allergy with the value 'Fish' and a Dietary Preference with the value 'Vegetarian'. Please confirm if either or both of these memories already exist in our database."}

>

In [5]:
print(result)

- Dietary Preference: Vegetarian
- Allergy: Fish


In [6]:
import asyncio
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

llm = ChatOpenAI()

def get_response(user_query, memories):

    template = """
    You are a helpful assistant. 
    Answer the following questions considering the user's question.
    You will also be provided a list of properties, preferences and restrictions correlating to the user that you should heavily consider
    when answering the user. These attributes are very important and should not be ignored

    User question: {user_question}
    properties, preferences and restrictions: {memories}
    """

    prompt = ChatPromptTemplate.from_template(template)

    llm = ChatOpenAI()
        
    chain = prompt | llm | StrOutputParser()
    
    return chain.invoke({
        "user_question": user_query,
        "memories": memories
    })


In [7]:
print(get_response(subsequent_request, result))

Certainly! Here is a sample vegetarian meal plan for you considering your dietary preference and allergy:

Breakfast:
- Greek yogurt with mixed berries and a sprinkle of granola
- Whole grain toast with avocado spread and sliced tomatoes

Lunch:
- Quinoa salad with roasted vegetables (bell peppers, zucchini, and cherry tomatoes) and feta cheese
- Lentil soup with a side of whole grain bread

Dinner:
- Stir-fried tofu with broccoli, carrots, and snow peas in a ginger soy sauce served over brown rice
- Caprese salad with fresh mozzarella, tomatoes, basil, and balsamic glaze

Snacks:
- Hummus with sliced cucumbers and carrots
- Mixed nuts and dried fruits

Remember to always check labels and ingredients to ensure there are no hidden fish products in the food you consume. Enjoy your vegetarian meal plan!


In [8]:
from memo_store import MemoStore

store = MemoStore(verbosity=0, reset=False, path_to_db_dir="./tmp/teachable_agent_db")
store.list_memos()


LOADING MEMORY FROM DISK
    Location = ./tmp/teachable_agent_db/uid_text_dict.pkl
LIST OF MEMOS
  ID: 1
    INPUT TEXT: Allergy
    OUTPUT TEXT: Fish
  ID: 2
    INPUT TEXT: Dietary Preference
    OUTPUT TEXT: Vegetarian


In [9]:
from tools import MemoryTools

mems = MemoryTools.retrieve_memories('meal plan')
print(mems)

['Vegetarian', 'Fish']
